## Mortgage Rate Model

This is the first model being built using the model building framework

### Packages, Function Imports, and Keys

In [1]:
from fredapi import Fred
import pandas as pd

In [2]:
fred = Fred(api_key='1e208e2d66ac6382c25f85524a5820cc')

### Model Details

In [3]:
# Horizon for prediction, in weeks. -- Daily models can come later
TIME_HORIZON = 1

### Data Intake

In [4]:
def download_fred_data_as_dataframe(series_code = str):
    '''Downloads and cleans FRED API call into dataframe.'''

    # Download the series using fredapi
    fred_series = fred.get_series(series_code)

    # Convert the Series into a DataFrame and move the index into a column
    df_raw = fred_series.reset_index()

    # Convert the series code to lowercase for the column name
    lowercase_series_code = series_code.lower()

    # Explicitly rename columns by position to avoid guessing the column name
    df_raw.columns = ["date", lowercase_series_code]

    # Convert date column to datetime
    df_raw['date'] = pd.to_datetime(df_raw['date'])

    return df_raw

In [5]:
# Intake mortgage rate time series
df_mortgage_rate_weekly = download_fred_data_as_dataframe('MORTGAGE30US')

print(df_mortgage_rate_weekly)

           date  mortgage30us
0    1971-04-02          7.33
1    1971-04-09          7.31
2    1971-04-16          7.31
3    1971-04-23          7.31
4    1971-04-30          7.29
...         ...           ...
2851 2025-11-20          6.26
2852 2025-11-26          6.23
2853 2025-12-04          6.19
2854 2025-12-11          6.22
2855 2025-12-18          6.21

[2856 rows x 2 columns]


In [6]:
# 10 Year Treasury Yield
df_ten_year_yield_daily = download_fred_data_as_dataframe('DGS10')

print(df_ten_year_yield_daily)

            date  dgs10
0     1962-01-02   4.06
1     1962-01-03   4.03
2     1962-01-04   3.99
3     1962-01-05   4.02
4     1962-01-08   4.03
...          ...    ...
16682 2025-12-11   4.14
16683 2025-12-12   4.19
16684 2025-12-15   4.18
16685 2025-12-16   4.15
16686 2025-12-17   4.16

[16687 rows x 2 columns]


In [7]:
# 2 Year Treasury Yield
df_two_year_yield_daily = download_fred_data_as_dataframe('DGS2')

print(df_two_year_yield_daily)

            date  dgs2
0     1976-06-01  7.26
1     1976-06-02  7.23
2     1976-06-03  7.22
3     1976-06-04  7.12
4     1976-06-07  7.09
...          ...   ...
12922 2025-12-11  3.52
12923 2025-12-12  3.52
12924 2025-12-15  3.51
12925 2025-12-16  3.48
12926 2025-12-17  3.49

[12927 rows x 2 columns]


In [8]:
# 10Y–2Y spread
df_ten_two_yield_spread_daily = download_fred_data_as_dataframe('T10Y2Y')

print(df_ten_two_yield_spread_daily)

            date  t10y2y
0     1976-06-01    0.68
1     1976-06-02    0.71
2     1976-06-03    0.70
3     1976-06-04    0.77
4     1976-06-07    0.79
...          ...     ...
12923 2025-12-12    0.67
12924 2025-12-15    0.67
12925 2025-12-16    0.67
12926 2025-12-17    0.67
12927 2025-12-18    0.66

[12928 rows x 2 columns]


In [9]:
# Effective Federal funds rate
df_federal_funds_rate_daily = download_fred_data_as_dataframe('DFF')

print(df_federal_funds_rate_daily)

            date   dff
0     1954-07-01  1.13
1     1954-07-02  1.25
2     1954-07-03  1.25
3     1954-07-04  1.25
4     1954-07-05  0.88
...          ...   ...
26098 2025-12-13  3.64
26099 2025-12-14  3.64
26100 2025-12-15  3.64
26101 2025-12-16  3.64
26102 2025-12-17  3.64

[26103 rows x 2 columns]


### Clean Data
Follow the formatting of FRED

### Create a dataframe with dates, then merge all data onto it

In [10]:
df_dates = pd.DataFrame({
    "date": pd.date_range(start="1900-01-01", end=pd.Timestamp.today(), freq="D")
})

### Merge

In [11]:
# List of all dataframes that contain data that will be in our model.
list_data_dfs = [
    df_mortgage_rate_weekly, 
    df_ten_year_yield_daily, 
    df_two_year_yield_daily, 
    df_ten_two_yield_spread_daily, 
    df_federal_funds_rate_daily]

In [12]:
def merge_dates_with_data(base_dates_dataframe, data_dataframe):
    # Merge a single data dataframe onto the base dates dataframe.
    # This function always returns a NEW dataframe and does not modify inputs.
    # An outer merge is used so no dates are lost.
    merged_dataframe = base_dates_dataframe.merge(
        data_dataframe,
        how="outer",
        on="date"
    )

    return merged_dataframe

In [13]:
# Start with the master date dataframe
df_model_data = df_dates.copy()

# Merge each dataframe one at a time (explicitly, no loop)
df_model_data = merge_dates_with_data(df_model_data, df_mortgage_rate_weekly)
df_model_data = merge_dates_with_data(df_model_data, df_ten_year_yield_daily)
df_model_data = merge_dates_with_data(df_model_data, df_two_year_yield_daily)
df_model_data = merge_dates_with_data(df_model_data, df_ten_two_yield_spread_daily)
df_model_data = merge_dates_with_data(df_model_data, df_federal_funds_rate_daily)

### Data Transforming

In [20]:
# Rename Target column
target_column_name = 'mortgage30us'

df_model_data = df_model_data.rename(columns={target_column_name : 'target'})

In [21]:
# Drop rows where the target is not present.
df_model_data.dropna(subset=["target"], inplace=True)

### Modelling